In [ ]:
import numpy as np
import numpy_financial as npf
from numba import jit
import time
import matplotlib.pyplot as plt

In [ ]:
np.random.normal(loc=0.0, scale=1, size=(3,2,4))

**Initialise parameters**
- $100mio USD IRS notional - client pays 6m libor, receives 6m fix
- 5Y tenor to maturity (hence 60 discrete months)
- current 6M USD Libor = 3\% (assumes USD Libor term structure is flat, LIBOR resets every 6M upon which IRS is revalued)

For PFE monte-carlo simulation model using Euler stoch discretisation
- assume no drift to interest rate
- assume volatility in next 5Y will average 15\%
- time step is 1 month or 1/12 year

In [ ]:
notional = 100
tenor = 5
T = tenor*12
reset_steps = 6
fixed_rate = 0.05

stoch_drift = 0
stoch_vol = 0.15
t_delta = 1/12

**Generate z shocks**
- 1000 pass of shocks drawn from normal distribution, assumes future risk is symmetrical and porportional to (endogenous) historical vol
- each pass has 60 business days ahead

In [ ]:
npass = 1000

shocks = np.random.normal(loc=0.0, scale=stoch_vol, size=(npass, T))
shocks[:,0] = 0
shocks.shape

Generate forward market factor path with simulated shocks

In [ ]:
rates = np.zeros(shocks.shape)
rates[:,0] = fixed_rate

Compare numba JIT speed improvement

In [ ]:
def sim_path(rates, shocks, npass, T, mu, sig, t) -> np.ndarray:
    for i in range(0,npass,1):
        for step in range(1,T,1):
            z_shock = shocks[i,step]
            rates[i,step] = rates[i,step-1] + rates[i,step-1]*(mu*t + sig*np.sqrt(t)*z_shock)
    return rates

@jit(nopython=True)
def osim_path(rates, shocks, npass, T, mu, sig, t) -> np.ndarray:
    for i in range(0,npass,1):
        for step in range(1,T,1):
            z_shock = shocks[i,step]
            rates[i,step] = rates[i,step-1] + rates[i,step-1]*(mu*t + sig*np.sqrt(t)*z_shock)
    return rates

In [ ]:
start = time.time()
rates = sim_path(rates, shocks, 
                 npass, T,
                 mu=stoch_drift, sig=stoch_vol, t=t_delta)
end = time.time()

print("Elapsed (with compilation) = {:.30f}".format(end - start))

In [ ]:
start = time.time()
rates = osim_path(rates, shocks, 
                 npass, T,
                 mu=stoch_drift, sig=stoch_vol, t=t_delta)
end = time.time()

print("Elapsed (with compilation) = {:.30f}".format(end - start))

In [ ]:
rates

In [ ]:
time_step = np.arange(0,T,1)
N = np.floor(((T-1)*np.ones(T) - time_step)/reset_steps)
N

In [ ]:
discount_remaining_tenor = -((( (T-1)*np.ones(T) - time_step)/reset_steps)+N)
discount_remaining_tenor

From bank perspective, bank's pays 6m fix/receive 6m Libor

-PV[(float/2,PV_N,notional*(fix-float)/2] * (1+float/2)^discount_remaining_tenor

In [ ]:
# npf.pv(rate, nper, pmt, fv=0, when='end')
def calc_IRS_NPV(notional, fixed_rate, rates, N, discount_remaining_tenor):
    npv = np.zeros(rates.shape)
    for i in range(0,npass,1):
        for step in range(0,T,1):
            float_rate = rates[i,step]
            nper, ndisc = N[step], discount_remaining_tenor[step]
            npv[i,step] = npf.pv(float_rate/2, nper, notional*(fixed_rate - float_rate)/2)*(1+float_rate/2)**ndisc
    return npv

In [ ]:
npv = calc_IRS_NPV(notional, fixed_rate, rates, N, discount_remaining_tenor)
npv[0]

In [ ]:
pfe = np.maximum(npv, np.zeros(rates.shape))
epe = np.percentile(pfe,99,axis=0)
epe

In [ ]:
plt.plot(epe)
plt.show()